In [1]:
import logging
#from benchmarks.stress_limiters import *

logging.basicConfig(
    format='%(asctime)s %(name)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

logger = logging.getLogger(__name__)


In [4]:
!docker run -d --name redis-test -p 6379:6379 redis:7


docker: Error response from daemon: Conflict. The container name "/redis-test" is already in use by container "a3eee2373b2ea343b46b216e638dedc888c54a0a49fa28943a3d26264005c311". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [2]:
import time
import asyncio
from datetime import datetime
from redis.asyncio import Redis
from pyrate_limiter import Duration, Rate, Limiter, RedisBucket

rate_limits = [Rate(3, Duration.SECOND * 3)]
redis = Redis(host="localhost")
bucket = await RedisBucket.init(rate_limits, redis, "test")

limiter = Limiter(bucket, raise_when_fail=True, max_delay=Duration.DAY, retry_until_max_delay=True)

async def task(name, weight):
    acquired = await limiter.try_acquire(name, weight)
    print(f"{datetime.now()} {name}: {weight}, {acquired=}")

start = time.time()
# Works good
for i in range(10):
    await task(str(i), 1)
print(f'Run 10 calls in {time.time() - start:,.2f} sec')

await asyncio.sleep(3)

# Doen't work
start = time.time()
await asyncio.gather(*[task(str(i), 1) for i in range(10)])
print(f'Run 10 calls in {time.time() - start:,.2f} sec');

2025-07-19 13:38:22.248871 0: 1, acquired=True
2025-07-19 13:38:22.251988 1: 1, acquired=True
2025-07-19 13:38:22.252490 2: 1, acquired=True
2025-07-19 13:38:25.313376 3: 1, acquired=True
2025-07-19 13:38:25.314969 4: 1, acquired=True
2025-07-19 13:38:25.315476 5: 1, acquired=True
2025-07-19 13:38:28.357963 6: 1, acquired=True
2025-07-19 13:38:28.358966 7: 1, acquired=True
2025-07-19 13:38:28.358966 8: 1, acquired=True
2025-07-19 13:38:31.404529 9: 1, acquired=True
Run 10 calls in 9.16 sec
2025-07-19 13:38:34.432072 1: 1, acquired=True
2025-07-19 13:38:34.433073 0: 1, acquired=True
2025-07-19 13:38:34.434072 2: 1, acquired=True
2025-07-19 13:38:37.496899 9: 1, acquired=True
2025-07-19 13:38:37.497897 3: 1, acquired=True
2025-07-19 13:38:37.497897 4: 1, acquired=True
2025-07-19 13:38:40.604443 8: 1, acquired=True
2025-07-19 13:38:40.604950 7: 1, acquired=True
2025-07-19 13:38:40.604950 5: 1, acquired=True
2025-07-19 13:38:43.746471 6: 1, acquired=True
Run 10 calls in 9.32 sec


NameError: name 'o' is not defined